
---


**Projet Dapyness - Estimation de ventes e-commerce** 


---

*L’objectif de ce projet est d’estimer l’évolution du volume de vente de produits vendus en ligne par un site e-commerce en utilisant les données de ces ventes.*

*Pour cela, nous étudions une table de données que nous nommons **df** lors de l'importation.*



*   Nous commençons notre étude par importer, analyser et nettoyer notre jeu de données. 
*   Ensuite, nous créons de nouvelles variables jugées utiles pour la suite de notre étude.
*   Nous analysons ensuite graphiquement nos données.
*   Et enfin, nous procédons à la modélisation de nos données pour espérer obtenir de bonnes prévisions de ventes.






L'ensemble des fichiers codes de ce projet sont décomposés en 5 parties :



*   01 - Analyse exploratoire 1/2 - Etude des différentes colonnes
*   **02 - Analyse exploratoire 2/2 - Etude des valeurs manquantes**
*   03 - création de nouvelles colonnes
*   04 - Etude graphique des variables et tests statistiques
*   05 - Modélisation - Time Series

In [ ]:
# Cellule d'imporation des packages nécessaires aux codes

#packages basiques
import pandas as pd
import numpy as np

#partie graphique
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from bokeh.plotting import figure, output_notebook, show
output_notebook()

#tests statistiques
import statsmodels.api 
from scipy.stats import pearsonr

#modélisation
#régression
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error

#modèles arima et sarima
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX



In [ ]:
#Importation du fichier csv chez Sanae 
df=pd.read_csv('data-estimation.csv', sep=';')

# **Etude et gestion des valeurs manquantes**





---


**Commentaires**



---

In [ ]:
"""
A propos des NaNs, nous réflechissons à 2 possibilités : 

Première possibilité : Suppression des NaN
data_clean = data.dropna(axis=0, how = 'any', subset =['Description','CustomerID'])

Deuxième possibilité : 
  a) Conservation des NaNs de la colonne 'Description' et les remplacer par une nouvelle valeur. Par exemple : "Non identifié" 
  b) Conservation des NaNs de la colonne 'CustomerID' et créer un nouvel identifiant client (soit le même pour tous les NaNs, soit un différent pour chaque NaNs)  
"""

In [ ]:
# détection des colonnes avec au moins une valeur manquante 
df.isna().any(axis=0)
# --> les colonnes description et customerID contienent des valeurs manquantes

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool

In [ ]:
# calcul du nombre de valeurs manquantes pour chaque colonne
df.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
# Affichage des lignes qui ont au moins une valeur manquante 
df[df.isna().any(axis=1)]

## *Etude des valeurs manquantes de la colonne Description*

In [ ]:
# Affichage des 20 premières entrées contenant un Na dans la colonne Description 
df[df[['Description' ]].isna().any(axis=1)].head(20)
# --> Remarque : lorsque la colonne description contient une valeur manquante, on a l'impression
# que la colonne customerID contient également une valeur manquante 
# --> On a aussi l'impression que le prix est égal à 0 
#hypothèse vérifiée lors du code suivant

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/01/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/01/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/01/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/01/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/01/2010 14:34,0.0,NaN,United Kingdom
1988,536550,85044,NaN,1,12/01/2010 14:34,0.0,NaN,United Kingdom
2024,536552,20950,NaN,1,12/01/2010 14:34,0.0,NaN,United Kingdom
2025,536553,37461,NaN,3,12/01/2010 14:35,0.0,NaN,United Kingdom
2026,536554,84670,NaN,23,12/01/2010 14:35,0.0,NaN,United Kingdom
2406,536589,21777,NaN,-10,12/01/2010 16:50,0.0,NaN,United Kingdom


In [ ]:
# code permettant de connaître les différentes valeurs de CustomerID lorsque la colonne Description contient une valeur manquante
df[df[['Description']].isna().any(axis=1)].CustomerID.unique()
# --> uniquement nan => lorsque la colonne description contient une valeur manquante, la colonne customerID également

array([nan])

In [ ]:
# code permettant de connaître les différentes valeurs de prix lorsque la colonne Description = NaN
df[df[['Description']].isna().any(axis=1)].UnitPrice.unique()
# --> prix = 0 pour toutes les fois où nous avons une description = Nan 

array([0.])

In [ ]:
# affichage des lignes lorsque Description = NaN, ordonnée par Quantity
df[df[['Description' ]].isna().any(axis=1)].sort_values(by='Quantity', ascending=True).head(10)
# Remarque : lorsque Description = NaN, et quantity < 0, InvoiceNo ne contient pas le caractère 'C' avant le numéro

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
341601,566768,16045,NaN,-3667,9/14/2011 17:53,0.0,NaN,United Kingdom
323458,565304,16259,NaN,-3167,09/02/2011 12:18,0.0,NaN,United Kingdom
270886,560600,18007,NaN,-2834,7/19/2011 17:04,0.0,NaN,United Kingdom
156350,550133,85099F,NaN,-1092,4/14/2011 13:49,0.0,NaN,United Kingdom
44863,540241,35957,NaN,-939,01/05/2011 15:17,0.0,NaN,United Kingdom
332823,566121,23306,NaN,-682,09/09/2011 10:56,0.0,NaN,United Kingdom
169680,551201,16015,NaN,-620,4/27/2011 11:33,0.0,NaN,United Kingdom
305607,563700,85106,NaN,-553,8/18/2011 13:55,0.0,NaN,United Kingdom
277041,561086,15036,NaN,-530,7/25/2011 9:32,0.0,NaN,United Kingdom
221960,556300,20735,NaN,-472,06/10/2011 10:18,0.0,NaN,United Kingdom


In [ ]:
# code permettant de connaître les différents pays pour lesquels nous avons une donnée manquante dans la col Description
df[df[['Description']].isna().any(axis=1)].Country.unique()
# --> donnée manquantes uniquement lorsque le pays est 'United Kingdom'

array(['United Kingdom'], dtype=object)

In [ ]:
"""
 En résumé : lorsque Description contient une valeur manquante :
--> on a égalament une valeur manquante dans colonne CustomerID
--> on a un prix nul
--> le seul pays présent est United Kingdom
--> lorsque quantity < 0, InvoiceNo non composé de la lettre 'C' : ne correpond pas à un retour 

==> Nous n'avons pas beaucoup d'informations sur ces données lorsque Description=NaN 
   ==> Nous décidons de supprimer ces données (1% de la table)
"""

## *Etude des valeurs manquantes de la colonnes CustomerID*

In [ ]:
#création d'une table avec les entrées pour lesquelles CustomerID = Nan
customers_nan = df[df[['CustomerID']].isna().any(axis=1)]

#suppression des lignes pour lesquelles description=Nan (car ce cas a été étudié plus haut)
customers_nan=customers_nan.dropna(axis=0,how='all', subset=['Description'])
customers_nan.head(10)

Etudions dans un premier temps cette table de clients non identifiés lorsque quantity < 0 

In [ ]:
#affichage de la table customers_nan lorsque la quantité est négative
customers_nan[customers_nan['Quantity']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7313,537032,21275,?,-30,12/03/2010 16:50,0.00,NaN,United Kingdom
11502,C537251,22429,ENAMEL MEASURING JUG CREAM,-2,12/06/2010 10:45,4.25,NaN,United Kingdom
11503,C537251,22620,4 TRADITIONAL SPINNING TOPS,-8,12/06/2010 10:45,1.25,NaN,United Kingdom
11504,C537251,21890,S/6 WOODEN SKITTLES IN COTTON BAG,-2,12/06/2010 10:45,2.95,NaN,United Kingdom
11505,C537251,22564,ALPHABET STENCIL CRAFT,-5,12/06/2010 10:45,1.25,NaN,United Kingdom
11506,C537251,21891,TRADITIONAL WOODEN SKIPPING ROPE,-3,12/06/2010 10:45,1.25,NaN,United Kingdom
11507,C537251,22747,POPPY'S PLAYHOUSE BATHROOM,-6,12/06/2010 10:45,2.10,NaN,United Kingdom
11508,C537251,22454,MEASURING TAPE BABUSHKA RED,-8,12/06/2010 10:45,2.95,NaN,United Kingdom
11509,C537251,22327,ROUND SNACK BOXES SET OF 4 SKULLS,-4,12/06/2010 10:45,2.95,NaN,United Kingdom
11510,C537251,21915,RED HARMONICA IN BOX,-4,12/06/2010 10:45,1.25,NaN,United Kingdom


In [ ]:
''' Commentaire : 
Il semblerait qu'il y ait des 'problèmes de commandes'. Nous pouvons voir cela dans la colonne Description
où l'on peut trouver des valeurs comme : ?, lost, missing,... 
--> Pour ces commandes 'problèmes', la facture semble ne pas être composée de la lettre 'C',
 ce qui renforce notre hypothèse '''

In [ ]:
# affichage des clients NaN lorsque la facture ne contient pas la lettre 'C' et quantity est négatif
customers_nan[-(customers_nan['InvoiceNo'].str.contains("C")) & (customers_nan['Quantity']<0)].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7313,537032,21275,?,-30,12/03/2010 16:50,0.0,NaN,United Kingdom
13217,537425,84968F,check,-20,12/06/2010 15:35,0.0,NaN,United Kingdom
13218,537426,84968E,check,-35,12/06/2010 15:36,0.0,NaN,United Kingdom
13264,537432,35833G,damages,-43,12/06/2010 16:10,0.0,NaN,United Kingdom
21338,538072,22423,faulty,-13,12/09/2010 14:10,0.0,NaN,United Kingdom


In [ ]:
# affichage des différentes valeurs de prix lorsque client NaN, quantity < 0 et facture ne contient pas la lettre 'C'
customers_nan[-(customers_nan['InvoiceNo'].str.contains("C")) & (customers_nan['Quantity']<0)].UnitPrice.unique()
#--> Nous trouvons uniquement UnitPrice=0 

array([0.])

In [ ]:
#affichage des différentes descriptions lorsque clients NaN, facture ne contient pas 'C' et quantity <0
customers_nan[-(customers_nan['InvoiceNo'].str.contains("C")) & (customers_nan['Quantity']<0)].Description.unique()
#Nous ne trouvons aucun produit dans cette liste de descirptions mais uniquement des informations ressemblant
#à des problèmes de commandes
#--> Notre hypothèse est vérifiée --> lorsque quantity < 0 et facture ne contient pas la lettre 'C' 
#les lignes correspondent à des probèmes de commandes

array(['?', 'check', 'damages', 'faulty', 'Dotcom sales',
       'reverse 21/5/10 adjustment', 'mouldy, thrown away.', 'counted',
       'Given away', 'Dotcom', 'label mix up', 'samples/damages',
       'thrown away', 'incorrectly made-thrown away.', 'showroom', 'MIA',
       'Dotcom set', 'wrongly sold as sets', 'Amazon sold sets',
       'dotcom sold sets', 'wrongly sold sets', '? sold as sets?',
       '?sold as sets?', 'Thrown away.', 'damages/display',
       'damaged stock', 'broken', 'throw away', 'wrong barcode (22467)',
       'wrong barcode', 'barcode problem', '?lost',
       "thrown away-can't sell.", "thrown away-can't sell", 'damages?',
       're dotcom quick fix.', "Dotcom sold in 6's", 'sold in set?',
       'cracked', 'sold as 22467', 'Damaged',
       'mystery! Only ever imported 1800',
       'MERCHANT CHANDLER CREDIT ERROR, STO', 'POSSIBLE DAMAGES OR LOST?',
       'damaged', 'DAMAGED', 'Display', 'Missing', 'wrong code?',
       'wrong code', 'adjust', 'crushed', 

In [ ]:
# affichage des clients NaN lorsque la facture contient la lettre 'C'
customers_nan[customers_nan['InvoiceNo'].str.contains("C")]
# Dans ce cas, les lignes semblent correspondre à des retours de commandes. Il ne semble pas y avoir de problème de commande

In [ ]:
# affichage des différentes descriptions lorsque client=NaN et facture 'C'
customers_nan[customers_nan['InvoiceNo'].str.contains("C")].Description.unique()
# nous avons uniquement des descriptions d'articles normales dans le cas où la facture est composée de la lettre 'C'
# aucune descriptions 'problème' --> notre hypothèse est vérifiée

Etudions la table des clients NaN lorsque quantity > 0

In [ ]:
# affichage des clients Nan avec quantity > 0
customers_nan[customers_nan['Quantity']>0]
#--> semblent correspondre à des ventes

In [ ]:
#affichage des clients Nans lorsque prix =0 et quantity > 0 
customers_nan[(customers_nan['UnitPrice'] == 0) & (customers_nan['Quantity']>0)]
# --> il semblerait qu'on ait également quelques descriptions "problèmes" lorsque la quantity est > 0

In [ ]:
#les différentes descriptions lorsque prix=0 et quantity>0
customers_nan[(customers_nan['UnitPrice'] == 0) & (customers_nan['Quantity']>0)].Description.unique()
# il y a bien quelques descriptions problems

In [ ]:
# Créons une liste contenant toutes les descriptions 'problèmes' que nous avons pu rencontrer


Description_problems = ['?', 'check', 'damages', 'faulty', 'Dotcom sales','reverse 21/5/10 adjustment',
                        'mouldy, thrown away.', 'counted','Given away', 'Dotcom', 'label mix up', 'samples/damages',
                        'thrown away', 'incorrectly made-thrown away.', 'showroom', 'MIA','Dotcom set',
                        'wrongly sold as sets', 'Amazon sold sets', 'dotcom sold sets', 'wrongly sold sets', 
                        '? sold as sets?','?sold as sets?', 'Thrown away.', 'damages/display','damaged stock',
                        'broken', 'throw away', 'wrong barcode (22467)','wrong barcode', 'barcode problem', '?lost',
                        "thrown away-can't sell.", "thrown away-can't sell", 'damages?','re dotcom quick fix.', 
                        "Dotcom sold in 6's", 'sold in set?','cracked', 'sold as 22467', 'Damaged',
                        'mystery! Only ever imported 1800','MERCHANT CHANDLER CREDIT ERROR, STO',
                        'POSSIBLE DAMAGES OR LOST?','damaged', 'DAMAGED', 'Display', 'Missing', 'wrong code?',
                        'wrong code', 'adjust', 'crushed', 'damages/showroom etc','samples', 
                        'damages/credits from ASOS.','Not rcvd in 10/11/2010 delivery', 'Thrown away-rusty',
                        'sold as set/6 by dotcom', 'wet/rusty', 'damages/dotcom?','smashed',
                        'reverse previous adjustment', 'incorrectly credited C550456 see 47', 'wet damaged',
                        'Water damaged', 'missing', 'sold as set on dotcom','sold as set on dotcom and amazon', 
                        'water damage', 'sold as set by dotcom', 'Printing smudges/thrown away',
                        'printing smudges/thrown away', 'found some more on shelf','Show Samples', 
                        'mix up with c', 'mouldy, unsaleable.','wrongly marked. 23343 in box',
                        'stock creditted wrongly', 'ebay','incorrectly put back into stock', 'Damages/samples',
                        'Sold as 1 on dotcom', 'taig adjust no stock','code mix up? 84930', '?display?',
                        'sold as 1', '?missing','crushed ctn', 'Crushed', 'temp adjustment', '??', 'test',
                        'OOPS ! adjustment', 'Dagamed','historic computer difference?....se',
                        'Incorrect stock entry.','incorrect stock entry.', 'wrongly coded-23343', 'stock check',
                        'crushed boxes', 'WET/MOULDY', "can't find", 'mouldy','Wet pallet-thrown away',
                        'adjustment', '20713 wrongly marked','re-adjustment', 'Breakages', '20713', 
                        'wrongly coded 20713','Damages', 'CHECK', 'Unsaleable, destroyed.', 'dotcom sales',
                        'damages wax', 'water damaged', 'Wrongly mrked had 85123a in box',
                        'wrongly marked carton 22804', 'missing?', 'wet rusty', '???lost','sold with wrong barcode',
                        'rusty thrown away', 'rusty throw away','dotcom', '?? missing', 'wet pallet', 
                        '????missing', '???missing','lost in space', 'wet?', 'lost??', '???', 'wet', 'wet boxes',
                        '????damages????', 'mixed up', 'lost', 'amazon', 'amazon sales', 'Adjustment',
                        'wrongly sold (22719) barcode','rcvd be air temp fix for dotcom sit',
                        'did  a credit  and did not tick ret','adjustment', 'returned','mailout ', 
                        'mailout', 'on cargo order','incorrectly credited C550456 see 47',
                        'to push order througha s stock was ','FOUND', 'came coded as 20713',
                        'alan hodge cant mamage this section', 'dotcom', 'test', 'taig adjust',
                        'allocate stock for dotcom orders ta','add stock to allocate online orders',
                        'for online retail orders','Amazon', 'found box', 'damaged', 'Found in w/hse',
                        'website fixed', 'Lighthouse Trading zero invc incorr',
                        'michel oops', 'wrongly coded 20713', 'Had been put aside.',
                        'Sale error', 'Amazon Adjustment', 'wrongly marked 23343',
                        'Marked as 23343', 'wrongly coded 23343', 'Found by jackie',
                        'check', 'wrongly marked', 'had been put aside', 'amazon adjust',
                        'dotcomstock', 'John Lewis', 'dotcom adjust', 'check?']


In [ ]:
# Vérifions que ces descriptions problèmes ne concernent que les clients non identifiés (dans la table df)
df[df.Description.isin(Description_problems)].CustomerID.unique()
# --> uniquement client NaN donc hyptohèse vérifiée

array([nan])

In [ ]:
# Vérifions que lorsque l'on rencontre ces descriptions problèmes, le prix est nul (dans table df)
df[df.Description.isin(Description_problems)].UnitPrice.unique()
# --> uniquement prix =0

array([0.])

In [ ]:
# affichage des clients NaN lorsque le prix = 0
customers_nan[customers_nan['UnitPrice'] == 0]
# lorsque le prix est nul, nous pouvons avoir des quantity > 0 ou < 0
# nous n'avons pas que des descriptions 'problèmes' mais également des produits : 
# hypothèse : ces produits sont des articles non payés (cadeau ou offre)

In [ ]:
''' Résumé : 
1) Lorsque Quantity < 0 et facture non composée de la lettre 'C' : 
   --> nous avons uniquement des descriptions problèmes et prix =0 
2) Lorsque Quantity < 0 et facture 'C' :
   --> ces lignes semblent correspondre à des retours de commande avec remboursement 
3) Lorsque Quantity > 0 : 
   --> Nous retrouvons quelques descriptions problèmes avec prix = 0 
   --> le reste des commandes semblent correspondre à des ventes OU des cadeaux/offres (lorsque prix = 0)

==> décision concernant les NaN de CustomerID :
Nous décidons de ne pas supprimer ces données (25% de la table) 
et nous attriburons une valeur 'client non identifié' à chaque NaN
Nous allons également différencier toutes les commandes selon leur type : vente, retours, problème, ...
'''

## *Gestion des valeurs manquantes*



In [ ]:
#Remplacement des valeurs manquantes NaN de CustomerID par une catégorie "Client non identifié"
df['CustomerID'].fillna('Client non identifié', inplace =True)
#Vérification avec une ligne au hasard contenant NaN à l'origine sur la variable CustomerID
print(df.loc[541538])

InvoiceNo                             581498
StockCode                              85150
Description    LADIES & GENTLEMEN METAL SIGN
Quantity                                   1
InvoiceDate                 12/09/2011 10:26
UnitPrice                               4.96
CustomerID              Client non identifié
Country                       United Kingdom
Name: 541538, dtype: object


In [ ]:
#Suppression des lignes contenant une valeur manquante NaN dans la variable Description 
df= df.dropna(axis=0,how='any', subset=['Description'])
#Vérification que toutes les lignes ont bien été supprimées
df[df['Description'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540455 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    540455 non-null  object 
 1   StockCode    540455 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     540455 non-null  int64  
 4   InvoiceDate  540455 non-null  object 
 5   UnitPrice    540455 non-null  float64
 6   CustomerID   540455 non-null  object 
 7   Country      540455 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 37.1+ MB
